<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import re
import time

#1:使用正则表达之去除html标签
def filter_tags(htmlstr):
#先过滤CDATA
    re_cdata=re.compile('//<!\[CDATA\[[^>]*//\]\]>',re.I) #匹配CDATA
    re_script=re.compile('<\s*script[^>]*>[^<]*<\s*/\s*script\s*>',re.I)#Script
    re_style=re.compile('<\s*style[^>]*>[^<]*<\s*/\s*style\s*>',re.I)#style
    re_br=re.compile('<br\s*?/?>')#处理换行
    re_h=re.compile('</?\w+[^>]*>')#HTML标签
    re_comment=re.compile('<!--[^>]*-->')#HTML注释
    s=re_cdata.sub('',htmlstr)#去掉CDATA
    s=re_script.sub('',s) #去掉SCRIPT
    s=re_style.sub('',s)#去掉style
    s=re_br.sub('\n',s)#将br转换为换行
    s=re_h.sub('',s) #去掉HTML 标签
    s=re_comment.sub('',s)#去掉HTML注释
    s=s.replace('网页链接','')
    return s


headers = {'user-agent':
          'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; LCTE; rv:11.0) like Gecko'
          }
          
cookies = {'cookies':
           '_T_WM=45802660733; WEIBOCN_FROM=1110006030; MLOGIN=0; M_WEIBOCN_PARAMS=luicode%3D10000011%26lfid%3D2310022728041997_-_HOTMBLOG%26fid%3D1076032728041997%26uicode%3D10000011; XSRF-TOKEN=354868'
          }

url_1 = input ( "请输入微博博主的url:" + str() ) 


i = 1
count = 1
while True:
    url = url_1 + '&page=' + str(i)
    req = requests.get(url, headers=headers, cookies=cookies)
    text = req.text
    try:
        content = json.loads(text).get('data')
        cards = content.get('cards')  
        if (len(cards)>0):
            for j in range(len(cards)):
                card_type = cards[j].get('card_type')
                if (card_type == 9):
                    mblog = cards[j].get('mblog')
                    attitudes_count=mblog.get('attitudes_count')
                    comments_count=mblog.get('comments_count')
                    created_at=mblog.get('created_at')
                    reposts_count=mblog.get('reposts_count')
                    scheme=cards[j].get('scheme')
                    text1=mblog.get('text')
                    text2=filter_tags(text1)
                    print ( str(count) + "\n"+"发布时间："+str(created_at)+"\n"+"微博内容："+text2+"\n")
                    count = count + 1
            i = i + 1
            time.sleep(5)
        else:
            break 
    except Exception as e:
        print (e)
        pass    

In [ ]:
https://m.weibo.cn/u/2728041997?uid=2728041997&luicode=10000011&lfid=2310022728041997_-_HOTMBLOG

In [ ]:
# -*- coding: utf-8 -*-

import urllib.request
import json

#定义要爬取的微博大V的微博ID
id='2778292197'

#设置代理IP
proxy_addr="122.241.72.191:808"

#定义页面打开函数
def use_proxy(url,proxy_addr):
    req=urllib.request.Request(url)
    req.add_header("User-Agent","Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.221 Safari/537.36 SE 2.X MetaSr 1.0")
    proxy=urllib.request.ProxyHandler({'http':proxy_addr})
    opener=urllib.request.build_opener(proxy,urllib.request.HTTPHandler)
    urllib.request.install_opener(opener)
    data=urllib.request.urlopen(req).read().decode('utf-8','ignore')
    return data

#获取微博主页的containerid，爬取微博内容时需要此id
def get_containerid(url):
    data=use_proxy(url,proxy_addr)
    content=json.loads(data).get('data')
    for data in content.get('tabsInfo').get('tabs'):
        if(data.get('tab_type')=='weibo'):
            containerid=data.get('containerid')
    return containerid

#获取微博大V账号的用户基本信息，如：微博昵称、微博地址、微博头像、关注人数、粉丝数、性别、等级等
def get_userInfo(id):
    url='https://m.weibo.cn/api/container/getIndex?type=uid&value='+id
    data=use_proxy(url,proxy_addr)
    content=json.loads(data).get('data')
    profile_image_url=content.get('userInfo').get('profile_image_url')
    description=content.get('userInfo').get('description')
    profile_url=content.get('userInfo').get('profile_url')
    verified=content.get('userInfo').get('verified')
    guanzhu=content.get('userInfo').get('follow_count')
    name=content.get('userInfo').get('screen_name')
    fensi=content.get('userInfo').get('followers_count')
    gender=content.get('userInfo').get('gender')
    urank=content.get('userInfo').get('urank')
    print("微博昵称："+name+"\n"+"微博主页地址："+profile_url+"\n"+"微博头像地址："+profile_image_url+"\n"+"是否认证："+str(verified)+"\n"+"微博说明："+description+"\n"+"关注人数："+str(guanzhu)+"\n"+"粉丝数："+str(fensi)+"\n"+"性别："+gender+"\n"+"微博等级："+str(urank)+"\n")


#获取微博内容信息,并保存到文本中，内容包括：每条微博的内容、微博详情页面地址、点赞数、评论数、转发数等
def get_weibo(id,file):
    i=1
    while True:
        url='https://m.weibo.cn/api/container/getIndex?type=uid&value='+id
        weibo_url='https://m.weibo.cn/api/container/getIndex?type=uid&value='+id+'&containerid='+get_containerid(url)+'&page='+str(i)
        try:
            data=use_proxy(weibo_url,proxy_addr)
            content=json.loads(data).get('data')
            cards=content.get('cards')
            if(len(cards)>0):
                for j in range(len(cards)):
                    print("-----正在爬取第"+str(i)+"页，第"+str(j)+"条微博------")
                    card_type=cards[j].get('card_type')
                    if(card_type==9):
                        mblog=cards[j].get('mblog')
                        attitudes_count=mblog.get('attitudes_count')
                        comments_count=mblog.get('comments_count')
                        created_at=mblog.get('created_at')
                        reposts_count=mblog.get('reposts_count')
                        scheme=cards[j].get('scheme')
                        text=mblog.get('text')
                        with open(file,'a',encoding='utf-8') as fh:
                            fh.write("----第"+str(i)+"页，第"+str(j)+"条微博----"+"\n")
                            fh.write("微博地址："+str(scheme)+"\n"+"发布时间："+str(created_at)+"\n"+"微博内容："+text+"\n"+"点赞数："+str(attitudes_count)+"\n"+"评论数："+str(comments_count)+"\n"+"转发数："+str(reposts_count)+"\n")
                i+=1
            else:
                break
        except Exception as e:
            print(e)
            pass

if __name__=="__main__":
    file=id+".txt"
    get_userInfo(id)
    get_weibo(id,file)

微博昵称：北京地铁
微博主页地址：https://m.weibo.cn/u/2778292197?uid=2778292197&luicode=10000011&lfid=1005052778292197
微博头像地址：https://tva4.sinaimg.cn/crop.0.0.180.180.180/a5995fe5jw1e8qgp5bmzyj2050050aa8.jpg?KID=imgbed,tva&Expires=1585312082&ssig=AXa5E7OCPH
是否认证：True
微博说明：北京市地铁运营有限公司官方微博
关注人数：518
粉丝数：4879363
性别：f
微博等级：48

-----正在爬取第1页，第0条微博------
-----正在爬取第1页，第1条微博------
-----正在爬取第1页，第2条微博------
-----正在爬取第1页，第3条微博------
-----正在爬取第1页，第4条微博------
-----正在爬取第1页，第5条微博------
-----正在爬取第1页，第6条微博------
-----正在爬取第1页，第7条微博------
-----正在爬取第1页，第8条微博------
-----正在爬取第1页，第9条微博------
-----正在爬取第1页，第10条微博------
-----正在爬取第1页，第11条微博------
-----正在爬取第2页，第0条微博------
-----正在爬取第2页，第1条微博------
-----正在爬取第2页，第2条微博------
-----正在爬取第2页，第3条微博------
-----正在爬取第2页，第4条微博------
-----正在爬取第2页，第5条微博------
-----正在爬取第2页，第6条微博------
-----正在爬取第2页，第7条微博------
-----正在爬取第2页，第8条微博------
-----正在爬取第2页，第9条微博------
-----正在爬取第3页，第0条微博------
-----正在爬取第3页，第1条微博------
-----正在爬取第3页，第2条微博------
-----正在爬取第3页，第3条微博------
-----正在爬取第3页，第4条微博------
-----正在爬取第3页，第5

-----正在爬取第32页，第0条微博------
-----正在爬取第32页，第1条微博------
-----正在爬取第32页，第2条微博------
-----正在爬取第32页，第3条微博------
-----正在爬取第32页，第4条微博------
-----正在爬取第32页，第5条微博------
-----正在爬取第32页，第6条微博------
-----正在爬取第32页，第7条微博------
-----正在爬取第32页，第8条微博------
-----正在爬取第32页，第9条微博------
-----正在爬取第33页，第0条微博------
-----正在爬取第33页，第1条微博------
-----正在爬取第33页，第2条微博------
-----正在爬取第33页，第3条微博------
-----正在爬取第33页，第4条微博------
-----正在爬取第33页，第5条微博------
-----正在爬取第33页，第6条微博------
-----正在爬取第33页，第7条微博------
-----正在爬取第33页，第8条微博------
-----正在爬取第33页，第9条微博------
-----正在爬取第34页，第0条微博------
-----正在爬取第34页，第1条微博------
-----正在爬取第34页，第2条微博------
-----正在爬取第34页，第3条微博------
-----正在爬取第34页，第4条微博------
-----正在爬取第34页，第5条微博------
-----正在爬取第34页，第6条微博------
-----正在爬取第34页，第7条微博------
-----正在爬取第34页，第8条微博------
-----正在爬取第34页，第9条微博------
-----正在爬取第35页，第0条微博------
-----正在爬取第35页，第1条微博------
-----正在爬取第35页，第2条微博------
-----正在爬取第35页，第3条微博------
-----正在爬取第35页，第4条微博------
-----正在爬取第35页，第5条微博------
-----正在爬取第35页，第6条微博------
-----正在爬取第35页，第7条微博------
-----正在爬取第35

In [ ]:
# -*- coding: utf-8 -*-

import urllib.request
import json

#定义要爬取的微博大V的微博ID
id='2778292197'

#设置代理IP
proxy_addr="122.241.72.191:808"

#定义页面打开函数
def use_proxy(url,proxy_addr):
    req=urllib.request.Request(url)
    req.add_header("User-Agent","Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.221 Safari/537.36 SE 2.X MetaSr 1.0")
    proxy=urllib.request.ProxyHandler({'http':proxy_addr})
    opener=urllib.request.build_opener(proxy,urllib.request.HTTPHandler)
    urllib.request.install_opener(opener)
    data=urllib.request.urlopen(req).read().decode('utf-8','ignore')
    return data

#获取微博主页的containerid，爬取微博内容时需要此id
def get_containerid(url):
    data=use_proxy(url,proxy_addr)
    content=json.loads(data).get('data')
    for data in content.get('tabsInfo').get('tabs'):
        if(data.get('tab_type')=='weibo'):
            containerid=data.get('containerid')
    return containerid

#获取微博大V账号的用户基本信息，如：微博昵称、微博地址、微博头像、关注人数、粉丝数、性别、等级等
def get_userInfo(id):
    url='https://m.weibo.cn/api/container/getIndex?type=uid&value='+id
    data=use_proxy(url,proxy_addr)
    content=json.loads(data).get('data')
    profile_image_url=content.get('userInfo').get('profile_image_url')
    description=content.get('userInfo').get('description')
    profile_url=content.get('userInfo').get('profile_url')
    verified=content.get('userInfo').get('verified')
    guanzhu=content.get('userInfo').get('follow_count')
    name=content.get('userInfo').get('screen_name')
    fensi=content.get('userInfo').get('followers_count')
    gender=content.get('userInfo').get('gender')
    urank=content.get('userInfo').get('urank')
    print("微博昵称："+name+"\n"+"微博主页地址："+profile_url+"\n"+"微博头像地址："+profile_image_url+"\n"+"是否认证："+str(verified)+"\n"+"微博说明："+description+"\n"+"关注人数："+str(guanzhu)+"\n"+"粉丝数："+str(fensi)+"\n"+"性别："+gender+"\n"+"微博等级："+str(urank)+"\n")


#获取微博内容信息,并保存到文本中，内容包括：每条微博的内容、微博详情页面地址、点赞数、评论数、转发数等
def get_weibo(id,file):
    i=1
    while True:
        url='https://m.weibo.cn/api/container/getIndex?type=uid&value='+id
        weibo_url='https://m.weibo.cn/api/container/getIndex?type=uid&value='+id+'&containerid='+get_containerid(url)+'&page='+str(i)
        try:
            data=use_proxy(weibo_url,proxy_addr)
            content=json.loads(data).get('data')
            cards=content.get('cards')
            if(len(cards)>0):
                for j in range(len(cards)):
                    print("-----正在爬取第"+str(i)+"页，第"+str(j)+"条微博------")
                    card_type=cards[j].get('card_type')
                    if(card_type==9):
                        mblog=cards[j].get('mblog')
                        attitudes_count=mblog.get('attitudes_count')
                        comments_count=mblog.get('comments_count')
                        created_at=mblog.get('created_at')
                        reposts_count=mblog.get('reposts_count')
                        scheme=cards[j].get('scheme')
                        text=mblog.get('text')
                        with open(file,'a',encoding='utf-8') as fh:
                            fh.write("----第"+str(i)+"页，第"+str(j)+"条微博----"+"\n")
                            fh.write("微博内容："+text+"\n")
                i+=1
            else:
                break
        except Exception as e:
            print(e)
            pass

if __name__=="__main__":
    file=id+"地铁.txt"
    #get_userInfo(id)
    get_weibo(id,file)

-----正在爬取第1页，第0条微博------
-----正在爬取第1页，第1条微博------
-----正在爬取第1页，第2条微博------
-----正在爬取第1页，第3条微博------
-----正在爬取第1页，第4条微博------
-----正在爬取第1页，第5条微博------
-----正在爬取第1页，第6条微博------
-----正在爬取第1页，第7条微博------
-----正在爬取第1页，第8条微博------
-----正在爬取第1页，第9条微博------
-----正在爬取第1页，第10条微博------
-----正在爬取第1页，第11条微博------
-----正在爬取第2页，第0条微博------
-----正在爬取第2页，第1条微博------
-----正在爬取第2页，第2条微博------
-----正在爬取第2页，第3条微博------
-----正在爬取第2页，第4条微博------
-----正在爬取第2页，第5条微博------
-----正在爬取第2页，第6条微博------
-----正在爬取第2页，第7条微博------
-----正在爬取第2页，第8条微博------
-----正在爬取第2页，第9条微博------
-----正在爬取第3页，第0条微博------
-----正在爬取第3页，第1条微博------
-----正在爬取第3页，第2条微博------
-----正在爬取第3页，第3条微博------
-----正在爬取第3页，第4条微博------
-----正在爬取第3页，第5条微博------
-----正在爬取第3页，第6条微博------
-----正在爬取第3页，第7条微博------
-----正在爬取第3页，第8条微博------
-----正在爬取第3页，第9条微博------
-----正在爬取第4页，第0条微博------
-----正在爬取第4页，第1条微博------
-----正在爬取第4页，第2条微博------
-----正在爬取第4页，第3条微博------
-----正在爬取第4页，第4条微博------
-----正在爬取第4页，第5条微博------
-----正在爬取第4页，第6条微博------
-----正在爬取第4页，第7条微博-----